In [1]:
#importing necessary libraries
import pandas as pd
import requests

In [2]:
#variables that will change each YEAR

#monetary values only
inflation_rate = .0676

#years you need
lastest_year = '2017/acs'
oldest_year = '2012'

#please change these as well
years = ['2012', '2017']

#ignore this
top_geogs = ['us:1', 'state:34']

In [3]:
#variables that will change each DATASET

#name of the variable you need and its margin of error (from https://api.census.gov/data/2017/acs/acs5/subject/variables.html)
api_variable = 'B25105_001E'
api_moe = 'B25105_001M'

#give it a name so that the column names line up right. Lowercase plz
api_name = 'housing_cost'

In [4]:
#this pulls the data you need for country and state columns
def create_top_dfs():
    my_data = {}
    for n in years:
        for d in top_geogs:
            url = 'https://api.census.gov/data/'+ n +'/acs/acs5?get='+ api_variable + ',NAME&for=' + d +  '&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7'
            r = requests.get(url)
            x = r.json()
            df = pd.DataFrame(x[1:],columns=x[0])
            df = df.replace(to_replace=['-555555555.0','-555555555',], value='')
            raw_result = pd.to_numeric(df[api_variable][0])
            if n == '2012':
                my_data[n + d] = raw_result * inflation_rate + raw_result
            my_data[n + d] = raw_result
    return my_data

In [5]:
top_df_dict = create_top_dfs()

#set the absolute values you will use later on
us_previous = top_df_dict['2012us:1']
us_latest = top_df_dict['2017us:1']
nj_previous = top_df_dict['2012state:34']
nj_latest = top_df_dict['2017state:34']

In [6]:
#pulls the data for the years and geography level you specify
def create_local_dfs(year, geography):
    df_list = []
    url = 'https://api.census.gov/data/'+ year +'/acs5?get=NAME,'+ api_variable +',' + api_moe + '&for=' + geography +  '&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7'
    print url
    r = requests.get(url)
    x = r.json()
    df = pd.DataFrame(x[1:], columns=x[0])
    #some null values will return as large negative numbers
    df = df.replace(to_replace=['-555555555.0','-555555555',], value='')
    #all variables returned as strings at first
    df[api_variable] = pd.to_numeric(df[api_variable], errors='coerce')
    df[api_moe] = pd.to_numeric(df[api_moe], errors='coerce')
    return df

In [7]:
#data cleaning function
def clean_local_dfs(year, label):
        muni = create_local_dfs(year, 'county%20subdivision:*')
        cty = create_local_dfs(year, 'county:*')
        #combines muni and county data
        local_df = muni.append(cty).reset_index().drop(labels='index', axis=1)
        local_df['county subdivision'] = local_df['county subdivision'].fillna('')
        #create FIPS code, since it initially returns as separate columns
        local_df['GEO.id2'] = local_df.state + local_df.county + local_df['county subdivision']
        local_df = local_df[[api_variable, api_moe, u'NAME',
                  u'GEO.id2']]
        #create "geography levels" for separate muni, county columns
        local_df['GEOLevel1'] = local_df['NAME'].str.split(', ', expand=True)[0]
        local_df['GEOLevel2'] = local_df['NAME'].str.split(', ', expand=True)[1]
        #calculate margin of errors's percent of value
        local_df['moe_pct'] = (local_df[api_moe] / local_df[api_variable]) * 100
        #rename from api variable to nice topic name
        local_df = local_df.rename(columns={api_variable: api_name, api_moe: api_name + '_moe', 'NAME':'GEOdisplaylabel'})
        local_df['GEOdisplaylabel'] = local_df['GEOdisplaylabel'].replace(to_replace=', New Jersey', value='', regex=True)
        if label == 'prev':
            local_df[api_name + '_prev_winf'] = local_df[api_name] * inflation_rate + local_df[api_name]
        return local_df

In [8]:
var_latest = clean_local_dfs(lastest_year, 'latest')
var_prev = clean_local_dfs(oldest_year, 'prev')

https://api.census.gov/data/2017/acs/acs5?get=NAME,B25105_001E,B25105_001M&for=county%20subdivision:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7
https://api.census.gov/data/2017/acs/acs5?get=NAME,B25105_001E,B25105_001M&for=county:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7
https://api.census.gov/data/2012/acs5?get=NAME,B25105_001E,B25105_001M&for=county%20subdivision:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7
https://api.census.gov/data/2012/acs5?get=NAME,B25105_001E,B25105_001M&for=county:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7


In [9]:
#Merge 
var_merge = pd.merge(var_latest, var_prev, left_on='GEO.id2', right_on='GEO.id2', how='outer', suffixes=('_latest', '_prev'))

In [10]:
#drop unneccessary columns and rename ones that need it
var_merge = var_merge.drop(['GEOdisplaylabel_prev', 'GEOLevel1_latest', 'GEOLevel2_latest'], axis=1)
var_merge.rename(columns={'GEOdisplaylabel_latest': 'GEOdisplaylabel', 'GEOLevel1_prev': 'GEOLevel1','GEOLevel2_prev':'GEOLevel2', 'GEO.id_latest': 'GEO.id', 'GEOdisplaylabel_long_x': 'GEOdisplaylabel_long'}, inplace=True)

In [11]:
#Take out County subdivisions not defined
var_merge = var_merge[var_merge['GEOdisplaylabel'].str.contains("County subdivisions not defined") == False]
var_merge = var_merge.sort_values(by=['GEO.id2'])

In [12]:
#Add NJ and Nationwide
var_merge['nj_' + api_name+'_latest'] = nj_latest
var_merge['nj_' + api_name+'_prev'] = nj_previous
var_merge['usa_' + api_name+'_latest'] = us_latest
var_merge['usa_' + api_name+'_prev'] = us_previous
var_merge[api_name+'_per_change'] = ((var_merge[api_name+'_latest'] - var_merge[api_name+'_prev']) / var_merge[api_name+'_prev']) * 100
var_merge['nj_' + api_name+'_change'] = ((var_merge['nj_' + api_name+'_latest'] - var_merge['nj_' + api_name+'_prev']) / var_merge['nj_' + api_name+'_prev']) * 100
var_merge['us_' + api_name+'_change'] = ((var_merge['usa_' + api_name+'_latest'] - var_merge['usa_' + api_name+'_prev']) / var_merge['usa_' + api_name+'_prev']) * 100

In [13]:
var_merge.columns

Index([u'housing_cost_latest', u'housing_cost_moe_latest', u'GEOdisplaylabel',
       u'GEO.id2', u'moe_pct_latest', u'housing_cost_prev',
       u'housing_cost_moe_prev', u'GEOLevel1', u'GEOLevel2', u'moe_pct_prev',
       u'housing_cost_prev_winf', u'nj_housing_cost_latest',
       u'nj_housing_cost_prev', u'usa_housing_cost_latest',
       u'usa_housing_cost_prev', u'housing_cost_per_change',
       u'nj_housing_cost_change', u'us_housing_cost_change'],
      dtype='object')

In [15]:
# re-ordering columns the way I want them

var_merge = var_merge[[ u'GEOdisplaylabel', u'GEO.id2',
        u'GEOLevel1', u'GEOLevel2',api_name + '_latest', api_name + '_moe_latest', 
        'moe_pct_latest',
        api_name + '_prev', api_name + '_moe_prev', 'moe_pct_prev', 'housing_cost_prev_winf',
       u'nj_' + api_name+'_latest', u'nj_' + api_name+'_prev',
       u'usa_' + api_name+'_latest', u'usa_' + api_name+'_prev',
       api_name + '_per_change', u'nj_' + api_name+'_change',
       u'us_' + api_name+'_change']]

In [16]:
var_merge.head()

,GEOdisplaylabel,GEO.id2,GEOLevel1,GEOLevel2,housing_cost_latest,housing_cost_moe_latest,moe_pct_latest,housing_cost_prev,housing_cost_moe_prev,moe_pct_prev,housing_cost_prev_winf,nj_housing_cost_latest,nj_housing_cost_prev,usa_housing_cost_latest,usa_housing_cost_prev,housing_cost_per_change,nj_housing_cost_change,us_housing_cost_change
577,Atlantic County,34001,Atlantic County,New Jersey,1355.0,19.0,1.402214,1312.0,20.0,1.524390,1400.6912,1548,1533,1037,1015,3.277439,0.978474,2.167488
98,"Absecon city, Atlantic County",3400100100,Absecon city,Atlantic County,1488.0,71.0,4.771505,1658.0,120.0,7.237636,1770.0808,1548,1533,1037,1015,-10.253317,0.978474,2.167488
121,"Atlantic City city, Atlantic County",3400102080,Atlantic City city,Atlantic County,937.0,24.0,2.561366,921.0,34.0,3.691640,983.2596,1548,1533,1037,1015,1.737242,0.978474,2.167488
109,"Brigantine city, Atlantic County",3400107810,Brigantine city,Atlantic County,1368.0,83.0,6.067251,1269.0,97.0,7.643814,1354.7844,1548,1533,1037,1015,7.801418,0.978474,2.167488
112,"Buena borough, Atlantic County",3400108680,Buena borough,Atlantic County,1097.0,166.0,15.132179,1151.0,106.0,9.209383,1228.8076,1548,1533,1037,1015,-4.691573,0.978474,2.167488


In [18]:
var_merge.to_csv('median_housing_costs_data_towns_clean.csv', header=True, index=False)